## Tool Installation

In [ ]:
# pip install bioinfokit

## Data Science Tools

In [ ]:
import bioinfokit as bik
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats

## GEX Matrix import and cell filtering

#### Add ST1 DBEC

In [ ]:
df_ST_1 = pd.read_csv("SampleTag01.csv", 
#                  sep='\t',
                 skiprows = 7, # For standard DBEC_MolsPerCell.csv file
                 low_memory=False)
df_ST_1

### CPM Norm

In [ ]:
from bioinfokit.analys import norm

nm = norm()
nm.cpm(df=df_ST_1)
cpm_df = nm.cpm_norm

cpm_df

### log2 Transform

In [ ]:
cpm_df_1 = cpm_df.applymap(lambda x: x+1)
cpm_df_log2 = cpm_df_1.applymap(np.log2)
cpm_df_log2.head()

### Gate population of interest

In [ ]:
from scipy.stats import gaussian_kde

# Markers
x = cpm_df_log2["CD4"]
y = cpm_df_log2["CD8A"]

# Calculate the point density
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots()
ax.scatter(x, y, c=z, s=10)
plt.show()

In [ ]:
gated_df = cpm_df_log2[cpm_df_log2['CD4'] < 2.5]
gated_df = gated_df[gated_df['CD8A'] > 2.0]

# # Markers
# x = gated_df["CD4:SK3|CD4|AHS0032|pAbO"]
# y = gated_df["CD8:RPA-T8|CD8A|AHS0027|pAbO"]

# # Calculate the point density
# xy = np.vstack([x,y])
# z = gaussian_kde(xy)(xy)

# fig, ax = plt.subplots()
# ax.scatter(x, y, c=z, s=10)
# plt.show()

## Retain Cell_Index

In [ ]:
gated_df_labels = gated_df.reset_index()

gated_df_labels = gated_df_labels[["Cell_Index"]].copy() 

gated_df_labels

#### https://support.10xgenomics.com/single-cell-vdj/software/pipelines/latest/algorithms/annotation#alignment 
#### Annotation is kind of the same for BD Rhapsody as it is for 10x Chromium

## TCR contigs import, gated cell label transfer and sorting

In [ ]:
df = pd.read_csv("Dominant_Contigs_AIRR.tsv", 
                 sep='\t',
                 low_memory=False)
df

## Remove Ig Calls

In [ ]:
df_TCR = df[df["locus"].str.contains("IGH") == False]
df_TCR = df_TCR[df_TCR["locus"].str.contains("IGK") == False]
df_TCR = df_TCR[df_TCR["locus"].str.contains("IGL") == False]
df_TCR

## Pick productive loci

In [ ]:
productive_df= df_TCR.loc[df_TCR['productive']==True]
productive_df

## Reconstruct Constant Fragments

In [ ]:
# TRAC

productive_df['sequence_aa'] = productive_df['sequence_aa'].str.replace('IQNPDPAVYQLRDSKSSDKSVCLFTDFD','IQNPDPAVYQLRDSKSSDKSVCLFTDFDSQTNVSQSKDSDVYITDKTVLDMRSMDFKSNSAVAWSNKSDFACANAFNNSIIPEDTFFPSPESSCDVKLVEKSFETDTNLNFQNLSVIGFRILLLKVAGFNLLMTLRLWSS')

# TRBC

productive_df['sequence_aa'] = productive_df['sequence_aa'].str.replace('PEVAVFEPSEA','PEVAVFEPSEAEISHTQKATLVCLATGFFPDHVELSWWVNGKEVHSGVSTDPQPLKEQPALNDSRYCLSSRLRVSATFWQNPRNHFRCQVQFYGLSENDEWTQDRAKPVTQIVSAEAWGRADCGFTSVSYQQGVLSATILYEILLGKATLYAVLVSALVLMAMVKRKDF')

productive_df

## Sort by locus

In [ ]:
productive_df.sort_values(by = 'locus', axis=0, ascending=True, inplace=True)
productive_df

## Subset colums of interest from the main df

In [ ]:
df_sort = productive_df[["cell_id", 'locus', 'sequence_aa','cdr3_aa']].copy() 

df_sort = df_sort.rename(columns={'cell_id': 'Cell_Index'})

df_sort.sort_values(by = 'Cell_Index', axis=0, ascending=True, inplace=True)

df_sort['sequence_aa'] = df_sort['sequence_aa'].apply(lambda x: x.rsplit('*', maxsplit=1)[-1])

df_sort['sequence_aa'] = df_sort['sequence_aa'].str.replace('^.*?[M]','M', regex=True)

df_sort.head(10)

## Pick TRA

In [ ]:
df_sort_A = df_sort[df_sort["locus"].str.contains("TRB") == False]
df_sort_A = df_sort_A[df_sort_A["locus"].str.contains("TRG") == False]
df_sort_A = df_sort_A[df_sort_A["locus"].str.contains("TRD") == False]
df_sort_A['locus_sequence_aa'] = df_sort_A[["locus", "sequence_aa"]].apply(
    lambda row: '_'.join(row.values.astype(str)), 
    axis=1)
df_sort_A.head()

## Pick TRB

In [ ]:
df_sort_B = df_sort[df_sort["locus"].str.contains("TRA") == False]
df_sort_B = df_sort_B[df_sort_B["locus"].str.contains("TRG") == False]
df_sort_B = df_sort_B[df_sort_B["locus"].str.contains("TRD") == False]
df_sort_B['locus_sequence_aa'] = df_sort_B[["locus", "sequence_aa"]].apply(
    lambda row: '_'.join(row.values.astype(str)), 
    axis=1)
df_sort_B.head()

## Merge TRA and TRB for single cells - locus_aaseq column

In [ ]:
df_AB = df_sort_A.append(df_sort_B)

df_AB.sort_values(by = 'locus', axis=0, ascending=True, inplace=True)

df_AB = df_AB[["Cell_Index", 'locus_sequence_aa']].copy() 

df_AB = df_AB.groupby("Cell_Index")['locus_sequence_aa'].apply(lambda x: '___'.join(x.astype(str))).reset_index()

df_AB = df_AB[df_AB["locus_sequence_aa"].str.contains("TRA") == True]
df_AB = df_AB[df_AB["locus_sequence_aa"].str.contains("TRB") == True]

df_AB

## Gated cell label transfer - AB TCR

In [ ]:
df_clonosort_AB = pd.merge(gated_df_labels, df_AB)

df_clonosort_AB.set_index(['Cell_Index'], inplace=True)

df_clonosort_AB

## Count AB clonotypes 

In [ ]:
df_AB_counts = df_clonosort_AB.groupby(df_clonosort_AB["locus_sequence_aa"].tolist(),as_index=False).size()

df_AB_counts.sort_values(by = 'size', axis=0, ascending=False, inplace=True)

df_AB_counts = df_AB_counts.rename(columns={'index': 'AB TCR', 'size' : 'Number of Cells full length TCR'})

df_AB_counts

## Visualise top 25 AB clonotypes' counts

In [ ]:
df_AB_vis = df_AB_counts.iloc[:75,:]

# df_AB_vis.set_index(['AB TCR AA-seq'], inplace=True)

df_AB_vis.plot(kind='bar',y='Number of Cells full length TCR', figsize=(15,10))

plt.savefig('Clonosort AB.png', bbox_inches='tight')
# plt.savefig('HER2-neu Clonosort AB.png', bbox_inches='tight')

plt.show()

## Pick TRA CDR3

In [ ]:
df_sort_A_cdr3 = df_sort[df_sort["locus"].str.contains("TRB") == False]
df_sort_A_cdr3 = df_sort_A_cdr3[df_sort_A_cdr3["locus"].str.contains("TRG") == False]
df_sort_A_cdr3 = df_sort_A_cdr3[df_sort_A_cdr3["locus"].str.contains("TRD") == False]
df_sort_A_cdr3['locus_cdr3_aa'] = df_sort_A_cdr3[["locus", "cdr3_aa"]].apply(
    lambda row: '_'.join(row.values.astype(str)), 
    axis=1)
df_sort_A_cdr3.head()

## Pick TRB CDR3

In [ ]:
df_sort_B_cdr3 = df_sort[df_sort["locus"].str.contains("TRA") == False]
df_sort_B_cdr3 = df_sort_B_cdr3[df_sort_B_cdr3["locus"].str.contains("TRG") == False]
df_sort_B_cdr3 = df_sort_B_cdr3[df_sort_B_cdr3["locus"].str.contains("TRD") == False]
df_sort_B_cdr3['locus_cdr3_aa'] = df_sort_B_cdr3[["locus", "cdr3_aa"]].apply(
    lambda row: '_'.join(row.values.astype(str)), 
    axis=1)
df_sort_B_cdr3.head()

## Merge TRA and TRB for single cells - locus_cdr3_aa column

In [ ]:
df_AB_cdr3 = df_sort_A_cdr3.append(df_sort_B_cdr3)

df_AB_cdr3.sort_values(by = 'locus', axis=0, ascending=True, inplace=True)

df_AB_cdr3 = df_AB_cdr3[["Cell_Index", 'locus_cdr3_aa']].copy() 

df_AB_cdr3 = df_AB_cdr3.groupby("Cell_Index")['locus_cdr3_aa'].apply(lambda x: '___'.join(x.astype(str))).reset_index()

df_AB_cdr3 = df_AB_cdr3[df_AB_cdr3["locus_cdr3_aa"].str.contains("TRA") == True]
df_AB_cdr3 = df_AB_cdr3[df_AB_cdr3["locus_cdr3_aa"].str.contains("TRB") == True]

df_AB_cdr3

## Gated cell label transfer - AB CDR3

In [ ]:
df_clonosort_AB_cdr3 = pd.merge(gated_df_labels, df_AB_cdr3)

df_clonosort_AB_cdr3.set_index(['Cell_Index'], inplace=True)

df_clonosort_AB_cdr3

In [ ]:
df_AB_counts_cdr3 = df_clonosort_AB_cdr3.groupby(df_clonosort_AB_cdr3["locus_cdr3_aa"].tolist(),as_index=False).size()

df_AB_counts_cdr3.sort_values(by = 'size', axis=0, ascending=False, inplace=True)

df_AB_counts_cdr3 = df_AB_counts_cdr3.rename(columns={'index': 'AB CDR3', 'size' : 'Number of Cells CDR3'})

df_AB_counts_cdr3.head(25)

In [ ]:
df_AB_vis_cdr3 = df_AB_counts_cdr3.iloc[:75,:]

df_AB_vis_cdr3.plot(kind='bar',y='Number of Cells CDR3', figsize=(15,10))

plt.savefig('CDR3sort AB.png', bbox_inches='tight')

plt.show()